In [24]:
from torchvision import transforms
from PIL import Image
import torch
from ultralytics import YOLO

In [2]:
model2 = YOLO('/home/ivan/university/maga/hakatons/27-29_10_23/guns_searcher/best.pt')
model_vgg = torch.load('/home/ivan/university/maga/hakatons/27-29_10_23/guns_searcher/model.pth')

In [3]:
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Resize(224),
        transforms.CenterCrop(224),
        
    ]
)

In [4]:
dict_class = {0:'another',1:'long_weapons',2:'short_weapons'}

In [61]:
results = model2(['/home/ivan/university/maga/hakatons/27-29_10_23/dataset/размеченные/1.jpg'])


0: 448x640 2 man_with_weapons, 2 weapons, 116.4ms
Speed: 12.8ms preprocess, 116.4ms inference, 0.6ms postprocess per image at shape (1, 3, 448, 640)


In [62]:
for r in results:
    r.names = {0: 'man_with_weapon', 1: 'man_without_weapon', 2: 'weapon',
                     3:'another',4:'long_weapons',5:'short_weapons'}
    img = Image.fromarray(r.orig_img[..., ::-1])
    
    for i,bord in enumerate(r.boxes):
        if bord.cls == 'weapon' or bord.cls == 2:
            
            data = transform(img.crop((
                             bord.xyxy[0][0].item(),
                             bord.xyxy[0][1].item(),
                             bord.xyxy[0][2].item(),
                             bord.xyxy[0][3].item()
                           )))
            data = torch.reshape(data,(1,*data.shape))
            result = model_vgg(data)
            label = str(dict_class[torch.max(result.data,1).indices.item()])
            print(label)
            try:
                r.boxes.cls[i]=3+torch.max(result.data,1).indices
            except RuntimeError:
                print(i,3+torch.max(result.data,1).indices, r.boxes.cls)
            
    
    im_array = r.plot()  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])
    im.show()# show image
    #im.save('results.jpg')  # save image
    

long_weapons
0 tensor([4]) tensor([4., 0., 2., 0.])
long_weapons
2 tensor([4]) tensor([4., 0., 4., 0.])
